In [10]:
import pandas as pd

In [11]:
house = pd.read_csv("./loc_data/house.csv")
pt = pd.read_csv("points_taipei.csv")
pnt = pd.read_csv("points_new_taipei.csv")

In [12]:
file_template = "year_score/{year}_{city}_{category}.csv"
cities = ["taipei", "new_taipei"]
categories = ["edu", "hos", "trans", "other"]
years = range(2019,2025)

# 初始化一個空字典來存儲所有的資料框
yearly_merge_t = {}
yearly_merge_nt = {}
yearly_house = {}

for year in years:
    year_merge_t = None
    for city in cities:
        city_year_merge = None
        for category in categories:
            file_path = file_template.format(city=city, category=category, year=year)
            try:
                df = pd.read_csv(file_path)
                # 如果 city_year_merge 尚未初始化，則將第一個資料集指定給 city_year_merge
                if city_year_merge is None:
                    city_year_merge = df
                else:
                    # 合併資料集
                    city_year_merge = pd.merge(city_year_merge, df, on=['x1', 'x2', 'y1', 'y2'], how='left', suffixes=('', f'_{category}'))
            except FileNotFoundError:
                print(f"檔案 {file_path} 不存在")
        
        if year_merge_t is None:
            year_merge_t = city_year_merge
        else:
            year_merge_nt = city_year_merge

    yearly_merge_t[year] = year_merge_t
    yearly_merge_nt[year] = year_merge_nt
    
    selected_columns = ["x1", "y1", "x2", "y2", f"房源數量_{year}", f"平均每坪單價_{year}", f"交易總價_{year}"]
    yearly_house[year] = house[selected_columns]


# 把每一年的 yearly_merge_t 和 yearly_house 合併
for year, df in yearly_merge_t.items():
    yearly_merge_t[year] = pd.merge(df, yearly_house[year], on=['x1', 'y1', 'x2', 'y2'], how='left')

for year, df in yearly_merge_nt.items():
    yearly_merge_nt[year] = pd.merge(df, yearly_house[year], on=['x1', 'y1', 'x2', 'y2'], how='left')

In [14]:
# 把 unnamed:5 改為 district2
for year, df in yearly_merge_t.items():
    df.rename(columns={"Unnamed: 5": "district2"}, inplace=True)
    df.rename(columns={'Unnamed: 6':'district3'}, inplace=True)
    df.rename(columns={'district6':'district1'}, inplace=True)
    
for year, df in yearly_merge_nt.items():
    df.rename(columns={"Unnamed: 5": "district2"}, inplace=True)
    df.rename(columns={'Unnamed: 6':'district3'}, inplace=True)
    df.rename(columns={'district6':'district1'}, inplace=True)


# 把平均每坪單價空值補0
for year, df in yearly_merge_t.items():
    df[f"平均每坪單價_{year}"] = df[f"平均每坪單價_{year}"].fillna(0)

for year, df in yearly_merge_nt.items():
    df[f"平均每坪單價_{year}"] = df[f"平均每坪單價_{year}"].fillna(0) 


# concat 不同城市的資料集
all_merge = {}
for year in years:
    all_merge[year] = pd.concat([yearly_merge_t[year], yearly_merge_nt[year]], ignore_index=True)

# 寫入檔案
for year, df in all_merge.items():
    df.to_csv(f"year_score/{year}_all_merge.csv", index=False)
for year, df in yearly_merge_t.items():
    df.to_csv(f"year_score/{year}_merge_taipei.csv", index=False)
for year, df in yearly_merge_nt.items():
    df.to_csv(f"year_score/{year}_merge_new_taipei.csv", index=False)